In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # only display errors and warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import matplotlib.pyplot as plt

import sys
sys.path.append('/lhome/ific/a/aamerio/github/MixtureEM/src/MixtureEM')

from MixtureEM import MixtureEM

2024-04-22 21:50:49.654452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
seed = 42
# set seed for tensorflow
tf.random.set_seed(seed)

In [3]:
dist1 = tfd.Normal(loc=1., scale=1.)
dist2 = tfd.Normal(loc=5., scale=1.)
p = [0.4, 0.6]

dist = tfp.distributions.Mixture(
    cat=tfp.distributions.Categorical(probs=p),
    components=[dist1, dist2]
)

data = dist.sample(1000)

2024-04-22 21:50:53.616850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
mu0 = tf.Variable([0.], dtype=tf.float32, name="mu0")
sigma0 = tf.Variable([0.5], dtype=tf.float32, name="sigma0")

mu1 = tf.Variable([10.], dtype=tf.float32, name="mu1")
sigma1 = tf.Variable([0.5], dtype=tf.float32, name="sigma1")

model0 = tfd.Normal(loc=mu0, scale=sigma0)
model1 = tfd.Normal(loc=mu1, scale=sigma1)

model = MixtureEM(initial_probs=[0.51, 0.49],
                  components=[model0, model1])


In [5]:
optimizer_EM = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [6]:
model.fit(data, optimizer_EM, algo="EM",opt_steps=100, likelihood_opt_steps=500, lr=1e-3)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:38<00:00,  2.57it/s, Log Likelihood: -2044.5608]


In [7]:
# optimizer_GD = tf.keras.optimizers.Adam(learning_rate=1e-8)
# #%% 
# model.fit(data, optimizer_GD, algo="GD",max_steps=1000)


In [8]:
for var in model.dist_variables:
    print(var.name, " ",var.numpy())

print("probs: ", model.probs_parameter.numpy())

mu0:0   [0.8701158]
sigma0:0   [0.9715155]
mu1:0   [4.9236546]
sigma1:0   [1.0782539]
probs:  [0.35337815 0.6466221 ]
